In [1]:
!conda install pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia -y

In [1]:
!pip install  -U bitsandbytes==0.42.0
!pip install  -U peft==0.8.2
!pip install  -U trl==0.7.10
!pip install  -U accelerate==0.27.1
!pip install  -U datasets==2.17.0
!pip install  -U transformers==4.38.1

  Using cached peft-0.8.2-py3-none-any.whl.metadata (25 kB)
Using cached peft-0.8.2-py3-none-any.whl (183 kB)
  Attempting uninstall: peft
    Found existing installation: peft 0.10.0
    Uninstalling peft-0.10.0:
      Successfully uninstalled peft-0.10.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmtuner 0.6.1 requires accelerate>=0.27.2, but you have accelerate 0.27.1 which is incompatible.
llmtuner 0.6.1 requires peft>=0.10.0, but you have peft 0.8.2 which is incompatible.


  Using cached trl-0.7.10-py3-none-any.whl.metadata (10 kB)
Using cached trl-0.7.10-py3-none-any.whl (150 kB)
  Attempting uninstall: trl
    Found existing installation: trl 0.8.1
    Uninstalling trl-0.8.1:
      Successfully uninstalled trl-0.8.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmtuner 0.6.1 requires accelerate>=0.27.2, but you have accelerate 0.27.1 which is incompatible.
llmtuner 0.6.1 requires peft>=0.10.0, but you have peft 0.8.2 which is incompatible.
llmtuner 0.6.1 requires trl>=0.8.1, but you have trl 0.7.10 which is incompatible.


   ---------------------------------------- 0.0/536.6 kB ? eta -:--:--
   --------------------------------------- 536.6/536.6 kB 35.1 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 2.17.1
    Uninstalling datasets-2.17.1:
      Successfully uninstalled datasets-2.17.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmtuner 0.6.1 requires accelerate>=0.27.2, but you have accelerate 0.27.1 which is incompatible.
llmtuner 0.6.1 requires peft>=0.10.0, but you have peft 0.8.2 which is incompatible.
llmtuner 0.6.1 requires trl>=0.8.1, but you have trl 0.7.10 which is incompatible.


  Using cached transformers-4.38.1-py3-none-any.whl.metadata (131 kB)
Using cached transformers-4.38.1-py3-none-any.whl (8.5 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.39.1
    Uninstalling transformers-4.39.1:
      Successfully uninstalled transformers-4.39.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmtuner 0.6.1 requires accelerate>=0.27.2, but you have accelerate 0.27.1 which is incompatible.
llmtuner 0.6.1 requires peft>=0.10.0, but you have peft 0.8.2 which is incompatible.
llmtuner 0.6.1 requires trl>=0.8.1, but you have trl 0.7.10 which is incompatible.


In [1]:
import torch
torch.cuda.is_available()

True

In [3]:
import torch,os
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GemmaTokenizer
import tqdm as notebook_tqdm


model_id = "beomi/gemma-ko-7b"

bnb_config = BitsAndBytesConfig(        # 4-bit quantization
    load_in_8bit=True,
    bnb_8bit_quant_type="nf8",
    bnb_8bit_compute_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained(model_id,padding_side = 'right')
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto",max_length=1024)

bin c:\Users\hyun0\anaconda3\envs\kdb3\lib\site-packages\bitsandbytes\libbitsandbytes_cuda121.dll


Loading checkpoint shards: 100%|██████████| 6/6 [00:03<00:00,  1.59it/s]


In [4]:
# CUDA 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [5]:
from peft import LoraConfig

lora_config = LoraConfig(      # lora configuration
    r=8,
    lora_alpha= 16,
    lora_dropout=0.1,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

In [6]:
import pandas as pd
from datasets import Dataset
# 데이터셋 불러오기
from datasets import load_dataset

dataset = load_dataset("maywell/koVast")

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['conversations'],
        num_rows: 684579
    })
})

In [8]:
dataset['train']['conversations'][0]

[{'from': 'user', 'value': '대학 입학 단계에서 의사소통 기회가 뭐야?'},
 {'from': 'gpt',
  'value': '대학 입학 단계에서의 의사소통 기회는 대학과 연결하고 다양한 수준의 학업과 교육 활동에 참여하기 위해 제공되는 기회를 말합니다. 이는 대학에 입학하고 처음 대학 생활을 시작할 때 학생들이 대학 내에서 자신을 표현하고 다른 사람들과 소통하는 기회를 제공합니다.'},
 {'from': 'user', 'value': '그럼 이런 의사소통 기회를 잘 활용하려면 어떻게 해야 해?'},
 {'from': 'gpt',
  'value': '이런 의사소통 기회를 잘 활용하기 위해서는 몇 가지 전략을 따르는 것이 좋습니다. 첫째, 강력하고 생산적인 관계를 형성하기 위해 대학 내에서 다양한 사람들과 교류하고 소통하는 것이 중요합니다. 둘째, 성공적인 대인관계를 위해서는 피드백을 제공하고 수락하는 것이 중요합니다. 이를 통해 서로의 의견을 이해하고 존중하며, 더 나은 관계를 형성할 수 있습니다.'},
 {'from': 'user', 'value': '전문적인 관점에서 소셜 미디어를 어떻게 활용해야 해?'},
 {'from': 'gpt',
  'value': '전문적인 관점에서 소셜 미디어를 활용하기 위해서는 몇 가지 팁을 따르는 것이 좋습니다. 첫째, 자신의 전문적인 입장을 강화하기 위해 소셜 미디어를 적절히 사용하는 것이 중요합니다. 이는 자신의 전문 분야에 관련된 정보를 공유하고, 전문적인 토론에 참여하는 것을 의미합니다. 둘째, 소셜 미디어를 통해 학업, 교외 활동 및 일자리 기회를 탐색하는 데 도움이 되는 정보를 찾아보는 것도 유용합니다. 이를 통해 자신의 경력 발전에 도움이 될 수 있는 기회를 발견할 수 있습니다.'},
 {'from': 'user', 'value': '서면 의사소통의 중요성은 구체적으로 뭐야?'},
 {'from': 'gpt',
  'value': '서면 의사소통의 중요성은 다양한 서신 및 문서를 만들고 

In [9]:
import wandb
secret_wandb = "155cbd25973f60b11ce57a095662e353e0cd6b80"
wandb.login(key = secret_wandb)
run = wandb.init(
    project='gemma-ko-7b-multiturn', 
    job_type="training", 
    anonymous="allow"
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hyun030508 (kdbwandb). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\hyun0\.netrc


In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['conversations'],
        num_rows: 684579
    })
})

In [11]:
new_model = "gemma-ko-7b-multiturn"

In [12]:
import transformers
from trl import SFTTrainer,DataCollatorForCompletionOnlyLM

def formatting_func(example): # example을 받아서 text로 변환
    output_texts = []
    for conversation in example['conversations']:
        for i in range(len(conversation)):
            if conversation[i]['from'] == 'user':
                question = conversation[i]['value']
                answer = conversation[i+1]['value'] if i+1 < len(conversation) and conversation[i+1]['from'] == 'gpt' else 'No answer'
                text = f"user: {question}\n model: {answer}"
                output_texts.append(text)
    return output_texts

response_template = "model:"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

In [13]:
train_subset = Dataset.from_dict({
    'conversations': dataset['train']['conversations'][:10000]
})

In [14]:
train_subset

Dataset({
    features: ['conversations'],
    num_rows: 10000
})

In [17]:
trainer = SFTTrainer(  # Supervised Fine-tuning Trainer
    model=model,
    train_dataset=train_subset,
    args=transformers.TrainingArguments( 
        output_dir=new_model,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        #warmup_steps=2,
        warmup_ratio=0.03,
        num_train_epochs=3,
        #max_steps=10,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        optim="paged_adamw_8bit",
        report_to="wandb",
        
    ),
    peft_config=lora_config,
    formatting_func=formatting_func,
    data_collator=collator,
    max_seq_length=2048
)
trainer.train()







  0%|          | 0/14844 [01:08<?, ?it/s]





  0%|          | 0/14844 [00:00<?, ?it/s]c:\Users\hyun0\anaconda3\envs\kdb3\lib\site-packages\trl\trainer\utils.py:129: UserWarning: Could not find response key `model:` in the following instance: <bos>user: 그럼 뉴스 보도가 사회에 미치는 긍정적인 영향과 부정적인 영향 중에 어느 쪽이 더 큼?
 model: 극단적 사건에 대한 뉴스 보도가 사회에 미치는 긍정적인 영향은 부정적인 영향보다 더 큽니다. 이는 뉴스 보도가 사회의 문제를 식별하고 대응하도록 도울 수 있으며, 상황 인식을 높이고 사람들이 스스로를 보호하는 방법을 배우도록 도울 수 있기 때문입니다. 또한, 이러한 보고서는 이러한 사건을 예방하기 위한 조치를 취하도록 도울 수 있습니다. This instance will be ignored in loss calculation. Note, if this happens often, consider increasing the `max_seq_length`.
  warnings.warn(
c:\Users\hyun0\anaconda3\envs\kdb3\lib\site-packages\trl\trainer\utils.py:129: UserWarning: Could not find response key `model:` in the following instance: <bos>user: 그럼 인지 심리학은 어떻게 발전했어?
 model: 인지 심리학은 비교적 최근에 발전한 학문 분야이지만, 매우 빠르게 성장하고 있습니다. 인지 심리학의 연구 결과는 교육, 임상, 경영 등 다양한 분야에서 응용되고 있으며, 이는 인지 심리학의 중요성과 유용성을 보여줍니다.<pad><pad><pad><pad><pad><pad><pad>

ValueError: Attempting to unscale FP16 gradients.

In [32]:
text = 'user : 전문적인 관점에서 소셜 미디어를 어떻게 활용해야 해? model :'
inputs = tokenizer(text, return_tensors="pt").to(device)

In [33]:
dataset['train']['conversations'][0]

[{'from': 'user', 'value': '대학 입학 단계에서 의사소통 기회가 뭐야?'},
 {'from': 'gpt',
  'value': '대학 입학 단계에서의 의사소통 기회는 대학과 연결하고 다양한 수준의 학업과 교육 활동에 참여하기 위해 제공되는 기회를 말합니다. 이는 대학에 입학하고 처음 대학 생활을 시작할 때 학생들이 대학 내에서 자신을 표현하고 다른 사람들과 소통하는 기회를 제공합니다.'},
 {'from': 'user', 'value': '그럼 이런 의사소통 기회를 잘 활용하려면 어떻게 해야 해?'},
 {'from': 'gpt',
  'value': '이런 의사소통 기회를 잘 활용하기 위해서는 몇 가지 전략을 따르는 것이 좋습니다. 첫째, 강력하고 생산적인 관계를 형성하기 위해 대학 내에서 다양한 사람들과 교류하고 소통하는 것이 중요합니다. 둘째, 성공적인 대인관계를 위해서는 피드백을 제공하고 수락하는 것이 중요합니다. 이를 통해 서로의 의견을 이해하고 존중하며, 더 나은 관계를 형성할 수 있습니다.'},
 {'from': 'user', 'value': '전문적인 관점에서 소셜 미디어를 어떻게 활용해야 해?'},
 {'from': 'gpt',
  'value': '전문적인 관점에서 소셜 미디어를 활용하기 위해서는 몇 가지 팁을 따르는 것이 좋습니다. 첫째, 자신의 전문적인 입장을 강화하기 위해 소셜 미디어를 적절히 사용하는 것이 중요합니다. 이는 자신의 전문 분야에 관련된 정보를 공유하고, 전문적인 토론에 참여하는 것을 의미합니다. 둘째, 소셜 미디어를 통해 학업, 교외 활동 및 일자리 기회를 탐색하는 데 도움이 되는 정보를 찾아보는 것도 유용합니다. 이를 통해 자신의 경력 발전에 도움이 될 수 있는 기회를 발견할 수 있습니다.'},
 {'from': 'user', 'value': '서면 의사소통의 중요성은 구체적으로 뭐야?'},
 {'from': 'gpt',
  'value': '서면 의사소통의 중요성은 다양한 서신 및 문서를 만들고 

In [34]:
outputs = model.generate(**inputs, max_new_tokens=512)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

user : 전문적인 관점에서 소셜 미디어를 어떻게 활용해야 해? model : 전문적인 관점에서 소셜 미디어를 활용하기 위해서는 소셜 미디어의 


In [ ]:
trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)

In [3]:
HUGGINGFACE_AUTH_TOKEN = '**'
MODEL_SAVE_HUB_PATH = f'Coldbrew9/Fine tuning gemma singleQ'
trainer.model.push_to_hub(
			MODEL_SAVE_HUB_PATH, 
			use_temp_dir=True, 
			use_auth_token=HUGGINGFACE_AUTH_TOKEN
)
trainer.tokenizer.push_to_hub(
			MODEL_SAVE_HUB_PATH, 
			use_temp_dir=True, 
			use_auth_token=HUGGINGFACE_AUTH_TOKEN
)

/home/sw_innov01/anaconda3/envs/gemma/lib/python3.11/site-packages/transformers/utils/hub.py:834: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/sw_innov01/anaconda3/envs/gemma/lib/python3.11/site-packages/transformers/integrations/peft.py:391: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(
adapter_model.safetensors: 100%|██████████████████████████████| 100M/100M [00:21<00:00, 4.67MB/s]
/home/sw_innov01/anaconda3/envs/gemma/lib/python3.11/site-packages/transformers/utils/hub.py:834: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
tokenizer.json: 100%|███████████████████████████████████████| 17.5M/17.5M [00:03<00:00, 4.88MB/s]


CommitInfo(commit_url='https://huggingface.co/Coldbrew9/Fine-tuning-gemma-doubleQ_v5_10Ep/commit/819e49bfe0d45ed2925693e28ee933bc7a601bc4', commit_message='Upload tokenizer', commit_description='', oid='819e49bfe0d45ed2925693e28ee933bc7a601bc4', pr_url=None, pr_revision=None, pr_num=None)